In [ ]:
"""
Created on Wed Aug 09 16:56 2023

Try to infer some Bayesian weights

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

READ IN CALVING FLUX

In [ ]:
inputpath_caving = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'
calving_flux = xr.open_dataset(inputpath_caving + 'calving_flux_rignot13.nc')

READ IN ATMO

model

In [ ]:
inputpath_atmo = '/bettik/kittelc/clara/'

scenario = 'historical'
melt_atmo = xr.open_dataset(inputpath_atmo+mod+'_SMB_'+scenario+'.nc').sel(Nisf=rignot_isf)
melt_atmo['time'] = melt_atmo['time'].dt.year

reanalysis

READ IN OCEAN

model

In [ ]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'quadratic_mixed_mean','quadratic_mixed_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNxsmall96',
              'NNsmall','NNlarge']

In [ ]:
## Melt outputpath
Gt_allmod_list = []
box1_allmod_list = []

scenario = 'historical'
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']:
#for mod in ['CNRM-CM6-1']:
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list = []
    for mparam in param_classic_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y']

    melt1D_list = []
    for mparam in param_NN_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')

    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    
    Gt_allmod_list.append(Gt_all.assign_coords({'model': mod}))

Gt_allmod = xr.concat(Gt_allmod_list, dim='model')
    

In [ ]:
mass_balance = Gt_allmod - melt_atmo['SMB'] + calving_flux

obs estimate

In [ ]:
inputpath_Paolo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'
data_Paolo = xr.open_dataset(inputpath_Paolo + 'ANT_G1920V01_IceShelfMelt.nc')

In [ ]:
inputpath_Paolo_processed = '/bettik/millanr/DATA_SERVER/ANTARCTICA/OCEANICE/DATASET_FINAL/'
integrated_Paolo = pd.read_csv(inputpath_Paolo_processed + 'Merged_Integrated_melt_rates.csv',index_col=0)
Paolo_Gt_ds = integrated_Paolo.to_xarray()

inputpath_Paolo_processed = '/bettik/millanr/DATA_SERVER/ANTARCTICA/OCEANICE/DATASET_FINAL/'
integrated_Paolo_unc = pd.read_csv(inputpath_Paolo_processed + 'Merged_Integrated_melt_err_rates.csv',index_col=0)
Paolo_Gt_unc_ds = integrated_Paolo_unc.to_xarray()


In [ ]:
year_list = ['1992'] + np.arange(1994,2015).astype(str).tolist()
merged_list = []
merged_list_unc = []
for vvar in year_list:
    merged_list.append(Paolo_Gt_ds[vvar].rename('melt_Gt').assign_coords({'time': int(vvar)}))
    merged_list_unc.append(Paolo_Gt_unc_ds[vvar].rename('melt_Gt_unc').assign_coords({'time': int(vvar)}))
merged_ds = xr.concat(merged_list, dim='time')
merged_unc_ds = xr.concat(merged_list_unc, dim='time')
merged_ds_both = xr.merge([merged_ds,merged_unc_ds])

In [ ]:
melt_Gt_list = []
for kisf in file_isf.Nisf:
    nname = file_isf['isf_name'].sel(Nisf=kisf).values
    if nname in merged_ds['Names']:
        melt_Gt_kisf = merged_ds_both.sel(Names=nname).squeeze().drop('Names')
    elif nname == 'Conger/Glenzer':
        melt_Gt_kisf = merged_ds_both.sel(Names='Conger_Glenzer').squeeze().drop('Names')
    elif nname == 'George VI':
        melt_Gt_kisf = merged_ds_both.sel(Names='George_VI').squeeze().drop('Names')
    elif str(nname)[0:7] == 'Larsen ':
        melt_Gt_kisf = merged_ds_both.sel(Names='Larsen'+str(nname)[7]).squeeze().drop('Names')
    elif nname == 'Prince Harald':
        melt_Gt_kisf = merged_ds_both.sel(Names='Prince_Harald').squeeze().drop('Names')
    elif nname == 'Moscow Univ.':    
        melt_Gt_kisf = merged_ds_both.sel(Names='Moscow_University').squeeze().drop('Names')
    elif nname == 'Roi Baudouin':
        melt_Gt_kisf = merged_ds_both.sel(Names='Baudouin').squeeze().drop('Names')
    elif nname == 'Wilma/Robert/Downer':
        melt_Gt_kisf = merged_ds_both.sel(Names='WilmaRobertDowner').squeeze().drop('Names')
    elif nname == 'Tracy Tremenchus':
        melt_Gt_kisf = merged_ds_both.sel(Names='Tracy_Tremenchus').squeeze().drop('Names')
    elif nname == 'Rayner/Thyer':
        melt_Gt_kisf = merged_ds_both.sel(Names='Rayner_Thyer').squeeze().drop('Names')
    elif nname == 'Pine Island':
        melt_Gt_kisf = merged_ds_both.sel(Names='Pine_Island').squeeze().drop('Names')
    elif nname == 'Stancomb Brunt':
        melt_Gt_kisf = merged_ds_both.sel(Names='Brunt_Stancomb').squeeze().drop('Names')
    elif nname == 'Edward VIII':
        melt_Gt_kisf = merged_ds_both.sel(Names='Edward_VIII').squeeze().drop('Names')
    elif nname == 'Vincennes Bay':
        melt_Gt_kisf = merged_ds_both.sel(Names='Vincennes_Bay').squeeze().drop('Names')
    elif nname == 'Filchner-Ronne':
        melt_Gt_kisf = merged_ds_both.sel(Names='Ronne').squeeze().drop('Names') + merged_ds_both.sel(Names='Filchner').squeeze().drop('Names')
    elif nname == 'Ross':
        melt_Gt_kisf = merged_ds_both.sel(Names='Ross_East').squeeze().drop('Names') + merged_ds_both.sel(Names='Ross_West').squeeze().drop('Names')
    elif nname == 'Wordie':
        melt_Gt_kisf = merged_ds_both.sel(Names='Wordie_(Harriott)').squeeze().drop('Names') + merged_ds_both.sel(Names='Wordie_(Harriott_Headland)'
                                                                               ).squeeze().drop('Names') + merged_ds_both.sel(Names='Wordie_(Cape_Jeremy)'
                                                                               ).squeeze().drop('Names') + merged_ds_both.sel(Names='Wordie_(Prospect)'
                                                                               ).squeeze().drop('Names') + merged_ds_both.sel(Names='Wordie_(Airy_Rotz_Seller)').squeeze().drop('Names')
    melt_Gt_list.append(melt_Gt_kisf.assign_coords({'Nisf': kisf}))

melt_Gt_all_Paolo = xr.concat(melt_Gt_list, dim='Nisf')
        


COMPUTE THE WEIGHT

In [ ]:

melt_Gt_all_Paolo

In [ ]:
year_list_int = (np.array(year_list)).astype(int)
diff_mod_obs = Gt_allmod.sel(time=year_list_int).mean('time') - melt_Gt_all_Paolo['melt_Gt'].mean('time') 

In [ ]:
sigma_obs = melt_Gt_all_Paolo['melt_Gt_unc'].max('time') 
sigma_mod = sigma_obs * 10

In [ ]:
s_j = np.exp(-((diff_mod_obs)**2/(sigma_obs**2 + sigma_mod**2)))

In [ ]:
weight = (s_j / (s_j.sum(['model','param'])))

In [ ]:
weight.sel(param=mparam,Nisf=kisf,model=mmod).values.item()

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = melt_Gt_all_Paolo['melt_Gt'].mean('time').sel(Nisf=kisf).values
    melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    for m,mmod in enumerate(Gt_allmod.model):
        for n,mparam in enumerate(Gt_allmod.param):

            ax[i].plot(np.arange(1992,2015), Gt_allmod.sel(param=mparam,time=range(1992,2015),Nisf=kisf,model=mmod), color=colors[m],
                        alpha=weight.sel(param=mparam,Nisf=kisf,model=mmod).values.item())



    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_historical_withobs_allmod.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = melt_Gt_all_Paolo['melt_Gt'].mean('time').sel(Nisf=kisf).values
    melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    ax[i].axhline(y=melt_obs_kisf,color='k', linewidth=3)
    ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    for m,mmod in enumerate(Gt_allmod.model):
        for n,mparam in enumerate(Gt_allmod.param):

            ax[i].axhline(y=Gt_allmod.sel(param=mparam,time=range(1992,2015),Nisf=kisf,model=mmod).mean('time'), color=colors[m],
                        alpha=weight.sel(param=mparam,Nisf=kisf,model=mmod).values.item() + 0.1) #x=np.arange(1992,2015), 



    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_historical_withobs_meanallmod.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = melt_Gt_all_Paolo['melt_Gt'].mean('time').sel(Nisf=kisf).values
    melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    ax[i].axhline(y=melt_obs_kisf,color='k', linewidth=3)
    ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    for m,mmod in enumerate(Gt_allmod.model):
        for n,mparam in enumerate(Gt_allmod.param):

            ax[i].axhline(y=Gt_allmod.sel(param=mparam,time=range(1992,2015),Nisf=kisf,model=mmod).mean('time'), color='blue',
                        alpha=weight.sel(param=mparam,Nisf=kisf,model=mmod).values.item() + 0.1) #x=np.arange(1992,2015), 



    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_historical_withobs_meanallmod.pdf')

In [ ]:
weight.stack({'flat_dim': ['model','param']})

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2)

ax={}

leg_hdl = []

i = 0

x = np.arange(len(weight.param))
y = np.arange(len(weight.model))

for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    ax[i].imshow(weight.sel(Nisf=kisf).values, cmap=plt.cm.Reds, vmin=0, vmax=0.05)   

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))

    if i > 55:        
        ax[i].set_xticks(x)
        ax[i].set_xticklabels(labels=weight.param.values, rotation=90)
    else:
        ax[i].set_xticks([])
        ax[i].set_xticklabels(labels=[])    
    
    if i in range(0,64,8):
        ax[i].set_yticks(y)
        ax[i].set_yticklabels(labels=weight.model.values)
    else:
        ax[i].set_yticks([])
        ax[i].set_yticklabels(labels=[])

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_historical_weights_heatmap.pdf')



In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = melt_Gt_all_Paolo['melt_Gt'].mean('time').sel(Nisf=kisf).values
    melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    for m,mmod in enumerate(Gt_allmod.model):
        for n,mparam in enumerate(Gt_allmod.param):

            ax[i].plot(np.arange(1992,2015), Gt_allmod.sel(param=mparam,time=range(1992,2015),Nisf=kisf,model=mmod), color='blue',
                        alpha=0.2)



    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_historical_withobs_allmod_withoutweight.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
#colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
#          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen','purple']


ax={}

leg_hdl = []



for m,mmod in enumerate(['CNRM-CM6-1']):
    f = plt.figure()
    f.set_size_inches(8.25*2, 8.25*2)
    
    i = 0

    for kisf in tqdm(sorted_isf_rignot):

        ax[i] = f.add_subplot(8,8,i+1)

        melt_obs_kisf = melt_Gt_all_Paolo['melt_Gt'].mean('time').sel(Nisf=kisf).values
        melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
        ax[i].axhline(y=melt_obs_kisf,color='k')
        #ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')

        for n,mparam in enumerate(Gt_allmod.param):

            ax[i].plot(np.arange(1992,2015), Gt_allmod.sel(param=mparam,time=range(1992,2015),Nisf=kisf,model=mmod), color=colors[n],
                        alpha=0.7)



        #if kisf == 23:
        #    ax[i].set_title('Tracy Tremenchus')
        #elif kisf == 24:
        #    ax[i].set_title('Conger/Glenzer')
        #elif kisf == 110:
        #    ax[i].set_title('Ekström')
        #else:
        ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
        #ax[i].set_xlim(0,60)
        #ax[i].axvline(x=30, c='k', linestyle='--')

        i = i+1

    #f.legend()
    f.subplots_adjust(bottom=0.05, wspace=0.1)

    f.tight_layout()
    sns.despine()
    plt.suptitle(mmod)
#plt.savefig(plot_path+'melt_compare_params_'+mod+'.pdf')

In [ ]:
melt_Gt_all_Paolo['melt_Gt_unc'].max('time')